<a href="https://colab.research.google.com/github/KIMNAMHYEON-Kpass/love-letter-lab/blob/main/notebooks/03_candidate_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 설치 및 환경설정

In [ ]:
# 설치
!pip install -q transformers accelerate sentencepiece datasets

# 임포트
import os, json, random, math, re
from pathlib import Path
from typing import List, Dict, Any

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 재현성
random.seed(42)
torch.manual_seed(42)

# 경로 준비
DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

RAW_PATH = DATA_DIR/"candidates_raw.json"
FILTERED_PATH = DATA_DIR/"candidates_filtered.json"

print({"data_dir": str(DATA_DIR), "raw": str(RAW_PATH), "filtered": str(FILTERED_PATH)})

# 2. 모델 로드

In [ ]:
# 한국어 감성/정서 분류 모델 예시 (모델은 상황에 맞게 교체 가능)
MODEL_NAME = "beomi/KcELECTRA-base"  # 예시: 토크나이저 용도로 불러와도 됨

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    # 감성 분류용 헤드가 포함된 모델로 교체 필요
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    model.eval()
    print({"model_loaded": MODEL_NAME, "device": device})
except Exception as e:
    print("모델 로딩 경고:", e)
    print("실제 감성분류 체크포인트로 교체해 주세요. (수업 모델/실습 모델 권장)")
    tokenizer = None
    model = None

# 3. 후보 생성

In [ ]:
# 데모용: 규칙/템플릿 기반 후보(10~20개) 생성
# 이후 LLM으로 대체할 때 이 함수만 교체하세요.
def generate_candidates_demo(name: str, relation: str, situation: str, tone: str, n: int = 20) -> List[str]:
    # 간단 템플릿과 어휘 풀(프로젝트 합의에 맞게 보강 가능)
    tone_words = {
        "달달": ["따뜻하게", "사근사근하게", "조심스럽게", "포근하게"],
        "시풍": ["은근히", "고즈넉히", "담담하게", "단정히"],
        "재치": ["가볍게", "살짝", "은근슬쩍", "재치있게"]
    }
    style = tone_words.get(tone, ["자연스럽게"])

    seeds = [
        f"{name}님, {situation} 속에서도 마음이 자꾸 {relation}로서 더 가까워지고 싶어집니다.",
        f"{name}님, {relation}로 지내며 알게 된 소중함을 {random.choice(style)} 전하고 싶어요.",
        f"{name}님께 조심스럽게 말해봅니다. {situation} 같은 순간에도 곁에서 응원하고 싶어요.",
        f"{name}님, 무리하지 않으셨으면 해요. {relation}로서 제가 옆에 있겠습니다.",
        f"{name}님과 조금 더 자주, {random.choice(['커피 한 잔','산책','짧은 대화'])}을 나누고 싶습니다."
    ]
    out = []
    for _ in range(n):
        s = random.choice(seeds)
        # 가벼운 변형
        s2 = s.replace("싶어집니다", random.choice(["바랍니다","원합니다","바라게 됩니다"]))
        if random.random() < 0.3:
            s2 += " 제 마음이 부담이 되지 않았으면 합니다."
        if random.random() < 0.3:
            s2 = s2.replace("응원하고 싶어요", "응원하고 싶습니다")
        out.append(s2)
    return out

# 예시 입력
name = "민수"
relation = "동기"
situation = "발표를 앞두고 긴장한 상황"
tone = "달달"

candidates_raw = generate_candidates_demo(name, relation, situation, tone, n=24)
print({"generated": len(candidates_raw)})
for i, c in enumerate(candidates_raw[:5], 1):
    print(f"{i:02d}. {c}")

# 4. 규칙 기반 필터링

In [ ]:
# 금칙(예시): 프로젝트 팀 합의에 따라 자유롭게 변경
PROFANITIES = [
    "씨발","좆","병신","꺼져","등신","죽어","지랄","sex","섹스","야한","폭력","협박"
]
# 과도한 사적인 추정/민감 표현 샘플
SENSITIVE = [
    "주민번호","주소","연봉","계좌","카드번호","숙박","만나자호텔"
]

# 형식 규칙
MAX_EXCLAIMS = 1    # 느낌표 최대 1회
ALLOW_EMOJI = False # 이모지 금지
MAX_LEN = 160       # 후보 문구 최대 길이(문자 기준) - 필요 시 조정
MIN_LEN = 20        # 최소 길이(너무 짧은 문구 제거 목적)

EMOJI_PATTERN = re.compile(r"[\U0001F600-\U0001F64F"
                           r"\U0001F300-\U0001F5FF"
                           r"\U0001F680-\U0001F6FF"
                           r"\U0001F1E0-\U0001F1FF]+", flags=re.UNICODE)

def contains_emoji(text: str) -> bool:
    return EMOJI_PATTERN.search(text) is not None

def count_exclaims(text: str) -> int:
    return text.count("!")

def rule_checks(text: str) -> Dict[str, Any]:
    reasons = []
    ok = True

    # 길이
    if not (MIN_LEN <= len(text) <= MAX_LEN):
        ok = False
        reasons.append({"rule":"length", "value":len(text), "min":MIN_LEN, "max":MAX_LEN})

    # 이모지
    if not ALLOW_EMOJI and contains_emoji(text):
        ok = False
        reasons.append({"rule":"emoji", "value":"found"})

    # 느낌표
    ex_cnt = count_exclaims(text)
    if ex_cnt > MAX_EXCLAIMS:
        ok = False
        reasons.append({"rule":"exclaim_limit", "value":ex_cnt, "max":MAX_EXCLAIMS})

    # 금칙
    prof_hits = [w for w in PROFANITIES if w in text]
    sens_hits = [w for w in SENSITIVE if w in text]
    if prof_hits:
        ok = False
        reasons.append({"rule":"profanity", "hits":prof_hits})
    if sens_hits:
        ok = False
        reasons.append({"rule":"sensitive", "hits":sens_hits})

    return {"ok": ok, "reasons": reasons}

# 5. 감성 필터링

In [ ]:
# 모델 아키텍처/라벨 매핑에 따라 점수 계산이 달라집니다.
def get_sentiment_score(text: str) -> Dict[str, Any]:
    """
    데모용 점수: 모델이 제대로 로딩되지 않으면 간단 휴리스틱으로 대체.
    """
    if tokenizer is None or model is None:
        # 휴리스틱 백업(긍부정 어휘 기반)
        pos_words = ["좋", "고맙", "따뜻", "응원", "함께", "설렘", "안심", "행복"]
        neg_words = ["미안", "불안", "걱정", "슬픔", "외로움", "싫", "후회"]
        pos = sum(w in text for w in pos_words)
        neg = sum(w in text for w in neg_words)
        total = max(1, pos+neg)
        score_pos = pos/total
        score_neg = neg/total
        label = "pos" if score_pos >= score_neg else "neg"
        return {"score_pos": float(score_pos), "score_neg": float(score_neg), "label": label, "mode": "heuristic"}
    else:
        with torch.no_grad():
            inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256).to(model.device)
            logits = model(**inputs).logits
            probs = torch.softmax(logits, dim=-1).squeeze().tolist()
            # 라벨 인덱스 매핑은 실제 모델 카드 문서 참고 필요
            if len(probs) == 2:
                score_neg, score_pos = probs[0], probs[1]
            else:
                score_pos = float(max(probs))
                score_neg = float(min(probs))
            label = "pos" if score_pos >= score_neg else "neg"
            return {"score_pos": float(score_pos), "score_neg": float(score_neg), "label": label, "mode": "model"}


# 6. 통과/탈락 분류

In [ ]:
def evaluate_candidate(text: str) -> Dict[str, Any]:
    rules = rule_checks(text)
    senti = get_sentiment_score(text)

    # 간단 통과 기준(예시): 규칙 통과 & 긍정 점수 >= 0.5
    passed = rules["ok"] and (senti["score_pos"] >= 0.5)

    return {
        "text": text,
        "passed": passed,
        "rules": rules,
        "sentiment": senti
    }

evaluated = [evaluate_candidate(t) for t in candidates_raw]
passed_items = [e for e in evaluated if e["passed"]]
failed_items = [e for e in evaluated if not e["passed"]]

print({"total": len(evaluated), "passed": len(passed_items), "failed": len(failed_items)})
for i, e in enumerate(passed_items[:5], 1):
    print(f"{i:02d}. {e['text']}  | pos={e['sentiment']['score_pos']:.2f}")

# 7. JSON 저장

In [ ]:
# 원본 후보 저장
raw_payload = {
    "meta": {"name": name, "relation": relation, "situation": situation, "tone": tone},
    "candidates": candidates_raw
}
with open(RAW_PATH, "w", encoding="utf-8") as f:
    json.dump(raw_payload, f, ensure_ascii=False, indent=2)

# 필터 결과 저장
filtered_payload = {
    "meta": {"name": name, "relation": relation, "situation": situation, "tone": tone},
    "selected": passed_items,
    "rejected": failed_items[:10]
}
with open(FILTERED_PATH, "w", encoding="utf-8") as f:
    json.dump(filtered_payload, f, ensure_ascii=False, indent=2)

print({"saved_raw": str(RAW_PATH), "saved_filtered": str(FILTERED_PATH)})

# 8. 검증

In [ ]:
# 노트북 자체 검증: 파일 존재 여부, 통과 항목≥1 여부를 딕셔너리로 print
print({
    "file_exists": FILTERED_PATH.exists(),
    "passed_items_gt_1": len(passed_items) >= 1,
})